In [4]:
from tqdm import tqdm
import os
import re
import pprint
import pandas as pd
import subprocess
import time

### Build dataset to find execution time of grep

In [ ]:
# read all files in the folder called regex

regex_list = []
desc_list = []
matcher_list = []
regex_complexity = []
regex_length = []
input_file = []
input_file_size = []

def calculate_pattern_complexity(patterns):
    complexity_scores = []
    for pattern in patterns:
        complexity = 0
        # Increase complexity score for each special regex character
        complexity += len(re.findall(r'[.*+?^${}()|[\]\\]', pattern))
        # Increase complexity score for pattern length
        complexity += len(pattern) / 10
        complexity_scores.append(complexity)
    # Return average complexity score
    return sum(complexity_scores) / len(complexity_scores) if complexity_scores else 0

def read_files():
    path = os.getcwd()
    path = path + "/regex/text.txt"
    with open(path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            regex = line.split("mmm")[0].strip()
            desc = line.split("mmm")[1].strip()
            matcher = desc.split("[")[1].split("]")[0]
            desc = desc.split("[")[0].strip()

            for m in ['-E', '-F', '-G', '-P', '-x']:
                for i in os.listdir("input_files/text_new/"):
                    regex_list.append(regex)
                    desc_list.append(desc)
                    matcher_list.append(m)
                    regex_length.append(len(regex))
                    regex_complexity.append(calculate_pattern_complexity([regex]))
                    filename_input = i
                    input_file.append(filename_input)
                    input_file_size.append(os.path.getsize("input_files/text_new/"+filename_input))
read_files()

df = pd.DataFrame()
df["regex"] = regex_list
# df["desc"] = desc_list
df["matcher"] = matcher_list
df["regex_length"] = regex_length
df["regex_complexity"] = regex_complexity
df["input_file"] = input_file
df["input_file_size (bytes)"] = input_file_size
df.head()

### find execution time using subprocess

In [ ]:

execution_times = []
s = set()

for index, row in tqdm(df.iterrows()):
    regex = row['regex']
    matcher = row['matcher']
    input_file = row['input_file']
    # print(regex, matcher, input_file)
    # print(os.listdir("."))
    timee = 0
    for i in range(3):
        error = False
        start_time = time.time()
        result = subprocess.run(['grep', matcher, regex, "input_files/text_new/"+input_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if result.returncode != 0 and result.stderr:
            # print(regex, matcher, input_file)
            # print(f"Error in grep execution: {result.stderr.decode('utf-8')}")
            s.add(f"Error in grep execution: {result.stderr.decode('utf-8')}" + "  " + regex + "  " + matcher)
            timee = None  # Mark as failed
            error = True
            continue
        output = result.stdout.decode('utf-8')
        if error:
            start_time = 0
            end_time = 0
        else:
            end_time = time.time()
        timee+=(end_time-start_time)*1000
    if timee is not None:
        execution_times.append(timee/3)
    else:
        execution_times.append("Error")

# df['execution_time (ms)'] = execution_times
df.head()
df["execution_time (ms)"] = execution_times

### Print the errors stored in set s

In [27]:
print(s)

{'Error in grep execution: grep: Invalid back reference\n  ^(?!.*(\\w)\\1{2})[a-zA-Z0-9]{8,}$  -E', 'Error in grep execution: grep: Invalid back reference\n  ^(?!.*(\\w)\\1{2})[a-zA-Z0-9]{8,}$  -x', 'Error in grep execution: grep: Invalid back reference\n  ^(?!.*(\\w)\\1{2})[a-zA-Z0-9]{8,}$  -G', 'Error in grep execution: grep: Invalid content of \\{\\}\n  (?<=^|\\s)(\\([^()]*\\)|\\[[^\\[\\]]*\\]|\\{[^{}]*\\})(?=\\s|$)  -x', 'Error in grep execution: grep: Invalid content of \\{\\}\n  (?<=^|\\s)(\\([^()]*\\)|\\[[^\\[\\]]*\\]|\\{[^{}]*\\})(?=\\s|$)  -G'}


### Save the dataset

In [15]:
df.to_csv("output/text_new.csv", index=False)

In [ ]:
df2 = pd.DataFrame()
df2["matcher"] = df["matcher"]
df2["regex_length"] = df["regex_length"]
df2["regex_complexity"] = df["regex_complexity"]
df2["input_file_size (bytes)"] = df["input_file_size (bytes)"]
df2["file_type"] = df["file_type"]
df2["execution_time (ms)"] = df["execution_time (ms)"]
df2

In [17]:
df2.to_csv("output/text_new_train.csv", index=False)